In [1]:
import scipy as sp
import numpy as np
from sklearn import decomposition
from sklearn import cluster
import math

In [2]:
%run load_dino slices-1k.out

In [ ]:
paths

In [3]:
np.shape(paths)
np.shape(paths)[0]
filtered_data = filter_paths(dinodata, .01)
indices = filtered_data[:,0]
paths = filtered_data[:,1:]
#reshape_to_distances(paths)
np.shape(paths)

(155, 144)

In [8]:
q = reshape_to_velocity(paths)
q[0,:] - paths[0,:]

array([-0.0585901, -0.1487129, -0.0759767, -0.0585901, -0.1487129,
       -0.0759767, -0.0585826, -0.1486685, -0.0759976, -0.0582716,
       -0.1484974, -0.0763963, -0.0598617, -0.1463887, -0.0773461,
       -0.063814 , -0.1457315, -0.0767439, -0.0656988, -0.1467215,
       -0.0757235, -0.0659368, -0.1471904, -0.0746798, -0.0651163,
       -0.1475822, -0.0727445, -0.0639793, -0.1475616, -0.0701149,
       -0.0623731, -0.1468268, -0.0668959, -0.0610947, -0.1457146,
       -0.0630014, -0.0606281, -0.1447931, -0.0588304, -0.0604028,
       -0.1441716, -0.0547372, -0.0603145, -0.1435453, -0.0514352,
       -0.0603717, -0.1431932, -0.0500978, -0.0603163, -0.1429447,
       -0.0491098, -0.0605114, -0.1426546, -0.0476489, -0.0608538,
       -0.1423971, -0.0459701, -0.0610724, -0.1423277, -0.0449402,
       -0.0611377, -0.1423492, -0.0446081, -0.061074 , -0.1424019,
       -0.0446774, -0.0610497, -0.1422918, -0.0450713, -0.0610861,
       -0.1419121, -0.0456675, -0.0610768, -0.1415571, -0.0461

In [ ]:
(c, t, tt) = rescale_similarity_measures(curvature, torsion, tortuosity)

In [ ]:
similarity(ns, 3, 0)

In [ ]:
np.sum(tt > 0.01)

In [9]:

(curvature, torsion, tortuosity) = get_similarity_measures(paths)
(c, t, tt) = rescale_similarity_measures(curvature, torsion, tortuosity)
sigs = generateSignatures(c, t, tt)

48
48


/Users/jtveite/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: RuntimeWarning: invalid value encountered in true_divide
/Users/jtveite/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:6: RuntimeWarning: invalid value encountered in true_divide


In [5]:
def get_similarity_measures(paths):
    velocity = reshape_to_velocity(paths)
    acceleration = reshape_to_acceleration(paths)
    v_cross_a = cross_matrix(velocity, acceleration)
    curvature = np.divide(magnitude_power(v_cross_a, 1), magnitude_power(velocity, 3))
    torsion = np.divide( dot_matrix(v_cross_a, reshape_to_derivative(acceleration)), magnitude_power(v_cross_a, 2))
    tortuosity = compute_tortuosity(paths)
    return (curvature, torsion, tortuosity)

def rescale_similarity_measures(curvature, torsion, tortuosity):
    for i in range(np.shape(curvature)[0]):
        for j in range(np.shape(curvature)[1]):
            if np.isnan(curvature[i,j]):
                curvature[i,j] = 0.0
    for i in range(np.shape(torsion)[0]):
        for j in range(np.shape(torsion)[1]):
            if np.isnan(torsion[i,j]):
                torsion[i,j] = 0.0
    c = curvature / np.max(curvature)
    t = torsion / np.max(torsion)
    tt =tortuosity / np.max(tortuosity)
    return (c, t, tt)

def generateSignatures(c, t, tt):
    out = np.zeros(np.shape(c));
    for i in range(np.shape(c)[0]):
        for j in range(np.shape(c)[1]):
            out[i,j] = c[i,j] + t[i,j] + tt[i]
    return out

def similarity(sigs, i, j):
    sim = 0
    for b in range(np.shape(sigs)[1]):
        num = math.pow(sigs[i,b] - sigs[j,b], 2)
        denom = sigs[i,b] + sigs[j,b]
        sim += (num / denom)
    return sim
    

In [6]:
def compute_tortuosity(paths):
    timesteps = int(math.floor(np.shape(paths)[1]/3))
    out = []
    for i in range(np.shape(paths)[0]):
        total_distance = get_path_length(paths[i,:])
        dx = paths[i, 0] - paths[i, timesteps * 3 - 3]
        dy = paths[i, 1] - paths[i, timesteps * 3 - 2]
        dz = paths[i, 2] - paths[i, timesteps * 3 - 1]
        shortest_distance = math.sqrt(dx ** 2 + dy ** 2 + dz ** 2)
        out.append(total_distance / shortest_distance)
    return out


def dot_matrix(a, b):
    timesteps = int(math.floor(np.shape(a)[1]/3))
    print(timesteps)
    out = np.zeros([np.shape(a)[0], timesteps])
    for i in range(np.shape(a)[0]):
        for j in range(timesteps):
            out[i, j] = a[i, 3*j] * b[i, 3*j] + a[i, 3*j+1] * b[i, 3*j+1] + a[i, 3*j+2] * b[i, 3*j+2]
    return out

def magnitude_power(matrix, power):
    timesteps = int(math.floor(np.shape(matrix)[1]/3))
    out = np.zeros([np.shape(matrix)[0], timesteps])
    for i in range(np.shape(matrix)[0]):
        for j in range(timesteps):
            base = matrix[i, 3*j] ** 2 + matrix[i, 3*j+1] ** 2 + matrix[i, 3*j+2] ** 2
            out[i, j] = math.pow(math.sqrt(base), power)
    return out
            

#computes cross product between matrix a and b, pointwise 
#THESE SHOULD BE THE SAME SIZE 
def cross_matrix(a,b):
    timesteps = int(math.floor(np.shape(a)[1]/3))
    print(timesteps)
    out = np.zeros(np.shape(a))
    for i in range(np.shape(a)[0]):
        for j in range(timesteps):
            out[i, 3*j] = a[i, 3*j+1] * b[i, 3*j+2] - a[i, 3*j+2] * b[i, 3*j+1]
            out[i, 3*j + 1] = a[i, 3*j+2] * b[i, 3*j+0] - a[i, 3*j+0] * b[i, 3*j+2]
            out[i, 3*j + 2] = a[i, 3*j+0] * b[i, 3*j+1] - a[i, 3*j+1] * b[i, 3*j+0]
    return out
    
#Computes something vaguely like a derivative 
def reshape_to_derivative(paths):
    timesteps = int(math.floor(np.shape(paths)[1] / 3))
    newpaths = np.copy(paths)
    for i in range(np.shape(paths)[0]):      
        newpaths[i, 0] = 0
        newpaths[i, 1] = 0
        newpaths[i, 2] = 0
        for j in range(timesteps - 1):
            newpaths[i,3*j+3] -= paths[i, 3*j]
            newpaths[i,3*j+4] -= paths[i, 3*j+1]
            newpaths[i,3*j+5] -= paths[i, 3*j+2]
    return newpaths

def reshape_to_velocity(paths):
    return reshape_to_derivative(paths)

def reshape_to_acceleration(paths):
    return reshape_to_derivative(reshape_to_derivative(paths))

In [7]:
def rebase_paths(paths, sigs, number_of_steps = 100):
    newpaths = np.empty((np.shape(paths)[0], number_of_steps))
    for i in range(np.shape(paths)[0]):
        if i % 1000 == 0:
            print(i)
        newpaths[i,:] = rebase_path(paths[i,:], sigs[i,:], number_of_steps)
    return newpaths

#Converts a path from list of points (at timesteps) to list of points (evenly distributed along the path)
def rebase_path(path, sigs,  number_of_steps = 100):
    total_length = get_path_length(path)
    timesteps = int(math.floor(np.shape(paths)[1] / 3))
    newpath = np.zeros(number_of_steps)
    current_timestep = 0
    current_distance = 0
    next_timestep_distance = 0
    last_timestep_distance = 0
    for i in range(number_of_steps):
        target_distance = i * 1.0 / number_of_steps * total_length
        next_timestep = current_timestep + 1
        total_value = 0
        steps_hit = 0
        #First we check to see which 2 points it's in between (current_timestep and next_timestep)
        while next_timestep_distance < target_distance:
            j = next_timestep
            if (j >= timesteps - 1):
                break
            next_timestep +=1
            last_timestep_distance = next_timestep_distance
            dx = path[3*j+3] - path[3*j]
            dy = path[3*j+4] - path[3*j+1]
            dz = path[3*j+5] - path[3*j+2]
            next_timestep_distance += math.sqrt(dx ** 2 + dy ** 2 + dz ** 2)
            total_value += sigs[j]
            steps_hit += 1
        current_timestep = next_timestep - 1
        if steps_hit > 0:
            newpath[i] = total_value / steps_hit
        else:
            newpath[i] = sigs[current_timestep]
        #Now figure out how far between them it is
        #j = current_timestep
        #a = sigs[j]
        #b = sigs[j+1]
        #try:
        #    lerp_ratio = (target_distance - last_timestep_distance) / (next_timestep_distance - last_timestep_distance)
        #except:
        #    lerp_ratio = 0
        #lerp_ratio = np.clip(lerp_ratio, 0, 1)
        #newpath[i] = (1 - lerp_ratio) * a + lerp_ratio * b
        #print(i,current_timestep,target_distance, lerp_ratio)
    return newpath

In [ ]:
#repaths = rebase_paths(paths,300)
a = np.ones([3,6])
b = np.array([0.5, 3, 1, 3, 4,6,3,4,5]*6).reshape([-1, 9])
c = compute_tortuosity(b)
c

In [10]:
target = 5000
p = target / np.shape(paths)[0]
p = np.clip(p, 0, 1)
print(p)
mask = np.random.choice([False, True], len(indices), p=[1-p, p])
ss_paths = paths[mask]
ss_indices = indices[mask]
ss_sigs = sigs[mask]
#ss_m = m[mask]
#np.shape(ss_m)
np.shape(ss_sigs)

1.0


(155, 48)

In [11]:
(curvature, torsion, tortuosity) = get_similarity_measures(ss_paths)
(c, t, tt) = rescale_similarity_measures(curvature, torsion, tortuosity)
sigs = generateSignatures(c, t, tt)

48
48


/Users/jtveite/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: RuntimeWarning: invalid value encountered in true_divide
/Users/jtveite/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:6: RuntimeWarning: invalid value encountered in true_divide


In [15]:
%timeit similarity(new_sigs, 5, 10)

10000 loops, best of 3: 109 µs per loop


In [14]:
new_sigs = rebase_paths(paths, sigs, 100)
new_sigs.shape

0


(155, 100)

In [ ]:
num_elements = sigs.shape[0]
simat = np.zeros([num_elements, num_elements])
for i in range(num_elements):
    if i % 50 == 0:
        print(i)
    for j in range(i, num_elements):
        val = similarity(new_sigs, i, j)
        simat[i,j] = val
        simat[j,i] = val

In [ ]:
agglom = cluster.AgglomerativeClustering(n_clusters=50, affinity='precomputed', linkage='complete')
labels = agglom.fit_predict(simat)

In [ ]:
agglom.children_

In [ ]:
labels

In [ ]:
np.histogram(labels, 20)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
plt.scatter(ss_m[:,0], ss_m[:,1], c=labels)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(ss_paths[:,0], ss_paths[:,1], ss_paths[:,2], s=10, c=labels)

In [ ]:
labelmap = {}
for i, label in enumerate(labels):
    if labelmap.get(label, []) == []:
        labelmap[label] = []
    labelmap[label].append(indices[i])

In [ ]:
labelmap[0][0]

In [ ]:
#Method for selecting 1 point from each cluster
outlabels = [values[0] for label, values in labelmap.items()]
outstr = '\n'.join([str(int(v)) for v in outlabels])
#print(outstr)

In [ ]:
#method for printing out clusterings (cluster id per line)
outlabels = [(v, l) for l, vs in labelmap.items() for v in vs ]#for clusterings
outstr = '\n'.join([str(int(v)) + ' ' + str(int(l+1)) for v, l in outlabels])

In [ ]:
#alternate method for printing clusterings (all members of a cluster on one line)
outlabels = [' '.join([str(int(i)) for i in items]) for _,items in labelmap.items()]
outstr = '\n'.join(outlabels)

In [ ]:
with open('sim-s68-20-50.clusters', 'w') as f:
    f.write(outstr)